## Двухфакторный дисперсионный анализ. Факторный анализ. Логистическая регрессия
### Задача 1
Объясняемая переменная y зависит от двух категориальных факторов A и B, причём фактор A имеет 3 уровня, а фактор B - 4 уровня. Результаты наблюдений приведены в таблице:
y = [[2.68, 3.29, 2.88, 4.45],
     [4.12, 4.96, 5.09, 5.22],
     [5.52, 4.50, 5.42, 5.29]]
С помощью двухфакторного дисперсионного анализа проверьте влияние каждого из факторов на переменную y.

In [5]:
import numpy as np
import pandas as pd
from scipy import stats

По каждому фактору проверяется нулевая гипотеза о равенстве средних значений на каждом уровне. Статистика для гипотезы о влиянии факторов $A$ и $B$:
$$F_a = \dfrac{\sigma_a^2}{\sigma_w^2}, \:\:\ F_b = \dfrac{\sigma_b^2}{\sigma_w^2}$$
Эти статистики имеют распределение Фишера с параметрами $k_{1a} = m - 1$, $k_{2a} = n - m$ и $k_{1b} = k - 1$, $k_{2b} = n - k$ соответственно, где
$m = 3$ - число уровней фактора $A$,
$k = 4$ - число уровней фактора $B$,
$n = 12$ - число наблюдений (m * k)
$$\sigma_a^2 = \dfrac{S_a^2}{m - 1}, \:\: \sigma_b^2 = \dfrac{S_b^2}{k - 1}, \:\: \sigma_w^2 = \dfrac{S_w^2}{(k - 1) (m - 1)}$$
$\sigma_a^2$ и $\sigma_b^2$ - оценки дисперсии между уровнями каждого фактора.
$\sigma_w^2$ - оценка дисперсии внутри всей выборки.
$$S_a^2 = k \cdot \displaystyle\sum_{i=1}^m \left( \overline{Y_{i \ast}} - \overline{Y} \right)^2, \:\: S_b^2 = m \cdot \displaystyle\sum_{j=1}^k \left( \overline{Y_{\ast j}} - \overline{Y} \right)^2, \:\: S_w^2 = \displaystyle\sum_{i=1}^m \displaystyle\sum_{j=1}^k \left( y_{ij} - \overline{Y_{i \ast}} - \overline{Y_{\ast j}} + \overline{Y} \right)^2$$
$\overline{Y}$ - среднее значение целевой переменной по всей выборке.
$\overline{Y_{i \ast}}$ - среднее значение переменной $y$ на $i$-м уровне фактора $A$.
$\overline{Y_{\ast j}}$ - среднее значение переменной $y$ на $j$-м уровне фактора $B$.

In [9]:
y = np.array([  [2.68, 3.29, 2.88, 4.45],
                [4.12, 4.96, 5.09, 5.22],
                [5.52, 4.50, 5.42, 5.29]])
m, k = y.shape
n = y.size
# cреднее значение по всей выборке
Y_mean = y.mean()
Y_mean
# среднее значение переменной  фактора  𝐴  по строкам
Y_A_mean = y.mean(axis=1)
Y_A_mean
# среднее значение переменной фактора  𝐵 по столбцам
Y_B_mean = y.mean(axis=0)
Y_B_mean
print(f'm={m},k={k},n={n}')
print(f'cреднее значение  : \t{Y_mean} \nсреднее значение А: \t{Y_A_mean} \nсреднее значение B: \t{(Y_B_mean)}')

m=3,k=4,n=12
cреднее значение  : 	4.451666666666666 
среднее значение А: 	[3.325  4.8475 5.1825] 
среднее значение B: 	[4.10666667 4.25       4.46333333 4.98666667]


Суммы квадратов отклонений: $S_a^2$ и $S_b^2$\
оценки дисперсии\
𝜎2𝑎 и 𝜎2𝑏 - оценки дисперсии между уровнями каждого фактора\
𝜎2𝑤 - оценка дисперсии внутри всей выборки.

In [13]:
S2_A = k * ((Y_A_mean -Y_mean)**2).sum()
S2_B = m * ((Y_B_mean -Y_mean)**2).sum()
S2_W = 0
for i in range(3):
    for j in range(4):
        S2_W += (y[i][j] - Y_A_mean[i] - Y_B_mean[j] + Y_mean)**2
simga2_A = S2_A / (m-1)
simga2_B = S2_B / (k-1)
sigma2_W = S2_W / ((k-1)*(m-1))
print(f'S2_A={S2_A}, S2_B={S2_B},S2_W={S2_W}')
print(f'simga2_A={simga2_A}, simga2_B={simga2_B}, sigma2_W={sigma2_W}')

S2_A=7.8407166666666654, S2_B=1.338166666666669,S2_W=1.9298833333333327
simga2_A=3.9203583333333327, simga2_B=0.4460555555555563, sigma2_W=0.32164722222222214


По каждому фактору проверяется нулевая гипотеза о равенстве средних значений на каждом уровне. Статистика для гипотезы о влиянии факторов 𝐴 и 𝐵 :

In [24]:
# Фактор А
# параметрами распределение Фишера
alpha = 0.05
k1_A = m - 1
k2_A = n - m
# Статистика
F_A = simga2_A / sigma2_W
# F-critical, уровень значимости α=0.05
F_A_critical = stats.f.ppf(1-alpha, k1_A, k2_A)
# Фактор B
alpha = 0.05
k1_B = k - 1
k2_B = n - k
F_B = simga2_B / sigma2_W
F_B_critical = stats.f.ppf(1-alpha, k1_A, k2_A)
print(f'Фактор А:F_A={round(F_A,4)}, F_A_critical={round(F_A_critical,4)}, F_A<F_A_critical->{F_A<F_A_critical}')
print(f'Фактор B: F_B={round(F_B,4)}, F_B_critical={round(F_B_critical,4)}, F_B<F_B_critical->{F_B<F_B_critical}')

Фактор А:F_A=12.1884, F_A_critical=4.2565, F_A<F_A_critical->False
Фактор B: F_B=1.3868, F_B_critical=4.2565, F_B<F_B_critical->True


$F_a \gt F_{a\:critical}$ Нулевая гипотеза о равенстве средних значений на уровнях фактора $A$ отвергается. Значение фактора $A$ влияет на результат.
$F_b \lt F_{b\:critical}$ Нулевая гипотеза о равенстве средних значений на уровнях фактора $B$ принимается. Значение фактора $В$ не влияет на результат.

### Задача 2

Тема "Прогнозирование медицинских выплат, оплачиваемых медицинской страховкой"

Данные датасета:
1)age: возраст основного бенефициара
2)sex: страховой подрядчик пол, женщина, мужчина
3)bmi: индекс массы тела, обеспечивающий понимание тела, веса, которые являются относительно высокими или низкими по отношению к росту, объективный индекс массы тела (кг / м ^ 2), использующий отношение роста к весу, в идеале от 18,5 до 24,9
4)children: Количество детей, охваченных медицинской страховкой / количество иждивенцев
5)smoker: курение
6)region: жилой район получателя в США, северо-восток, юго-восток, юго-запад, северо-запад.
7)charges: индивидуальные медицинские расходы, оплачиваемые медицинской страховкой

Я попытаюсь определить: 
1)предикторные переменные, анализируя набор данных 
2)определение корреляции между данными
3)определение регрессионных связей
для прогнозирования цены страхования.


In [10]:
data1 = pd.read_csv('insurance.csv', index_col=0, encoding='utf-8')
data = data1.copy()
data1.head(10)

,sex,bmi,children,smoker,region,charges
age,,,,,,
19,female,27.900,0,yes,southwest,16884.92400
18,male,33.770,1,no,southeast,1725.55230
28,male,33.000,3,no,southeast,4449.46200
33,male,22.705,0,no,northwest,21984.47061
32,male,28.880,0,no,northwest,3866.85520
31,female,25.740,0,no,southeast,3756.62160
46,female,33.440,1,no,southeast,8240.58960
37,female,27.740,3,no,northwest,7281.50560
37,male,29.830,2,no,northeast,6406.41070
